# JWT-based Authentication in FastAPI apps

## work flow is similar to in flaskAPI

1. **User Registration**: User creates account → Password gets hashed → Stored in database
   ```text
   Client                    FastAPI Server                Database
   |                            |                           |
   |-- POST /auth/register ---->|                           |
   |    {email, username,       |                           |
   |     password}              |                           |
   |                            |                           |
   |                            |-- Validate with Pydantic  |
   |                            |   (min length, digit)     |
   |                            |                           |
   |                            |-- hash_password() ------->|
   |                            |   "pass123"               |
   |                            |   ↓                       |
   |                            |   "$argon2id$..."         |
   |                            |                           |
   |                            |-- INSERT INTO users ----->|
   |                            |   (email, username,       |
   |                            |    hash_password)         |
   |                            |                           |
   |<-- 201 Created ------------|<-- User record created ---|
   |    {id, email, username}   |                           |
   ```
1. **User Login**: User sends credentials → Password verified → JWT token issued
   ```text
   Client                    FastAPI Server                Database
   |                            |                           |
   |-- POST /auth/login ------->|                           |
   |    {username, password}    |                           |
   |    (OAuth2 form format)    |                           |
   |                            |                           |
   |                            |-- SELECT * FROM users --->|
   |                            |   WHERE email = ?         |
   |                            |                           |
   |                            |<-- User record -----------|
   |                            |                           |
   |                            |-- verify_password()       |
   |                            |   Plain: "pass123"        |
   |                            |   Hash: "$argon2id$..."   |
   |                            |   ✓ Match!                |
   |                            |                           |
   |                            |-- create_access_token()   |
   |                            |   user_id: 1              |
   |                            |   exp: 30 min from now    |
   |                            |   ↓                       |
   |                            |   JWT token created       |
   |                            |                           |
   |<-- 200 OK -----------------|                           |
   |    {access_token: "eyJ..."} |                          |
   ```
1. **Accessing Protected Route**: User includes token → Token validated → User identified → Access granted
   ```text
   Client                    FastAPI Server                Database
   |                            |                           |
   |-- GET /notes ------------->|                           |
   |    Authorization: Bearer   |                           |
   |    eyJhbGci...             |                           |
   |                            |                           |
   |                            |-- OAuth2PasswordBearer    |
   |                            |   extracts token          |
   |                            |                           |
   |                            |-- decode_access_token()   |
   |                            |   Validates signature     |
   |                            |   Checks expiration       |
   |                            |   Extracts user_id: 1     |
   |                            |                           |
   |                            |-- SELECT * FROM users --->|
   |                            |   WHERE id = 1            |
   |                            |                           |
   |                            |<-- User object -----------|
   |                            |                           |
   |                            |-- Execute route handler   |
   |                            |   current_user available  |
   |                            |                           |
   |<-- 200 OK -----------------|                           |
   |    [user's notes]          |                           |
   ```

## Security
### Password Hashing with Argon2 algorithm (industry standard)

```python
from pwdlib import PasswordHash
pwd_hash = PasswordHash.recommended()

def hash_password(password: str) -> str:
    """Converts 'mypassword123' → '$argon2id$v=19$m=...'"""
    return pwd_hash.hash(password)

def verify_password(plain_password: str, hashed_password: str) -> bool:
    """Checks if plain password matches the stored hash"""
    return pwd_hash.verify(plain_password, hashed_password)
```

- Using `pwdlib` with recommended settings, instead of `werkzeug`.security for FlaskAPI
- Never storing plain text passwords

| Feature | Werkzeug | pwdlib |
|---------|----------|--------|
| **Default Algorithm** | PBKDF2(Password-Based Key Derivation Function 2) | Argon2id |
| **Security Level** | Good | Better |
| **Speed** | Moderate | Configurable |
| **Memory Hardness** | ❌ No | ✅ Yes (resistant to GPU attacks) |
| **Ecosystem** | Flask-focused | Framework-agnostic |
| **Modern Standards** | Older standard | Current best practice |
| **Easy to use** | ✅ Yes | ✅ Yes |

Argon2 requires significant memory to compute, making it resistant to:    
- GPU-based attacks
- ASIC-based attacks
- Parallel brute-force attempts

PBKDF2 (Werkzeug's default) only uses CPU, making it more vulnerable to specialized hardware attacks.

**Modern Security Standard**
2000s: PBKDF2 was the standard    
2013: bcrypt became popular     
2015: Argon2 wins Password Hashing Competition     
2024: Argon2id is the recommended algorithm    

- Argon2d: Resistant to GPU attacks (but vulnerable to side-channel)
- Argon2i: Resistant to side-channel attacks
- Argon2id: Hybrid (RECOMMENDED) - Best of both worlds

### Environment Variables
- Store secret keys in `.env file`, which is in the project root directory, same as the main.py
- for this project, add `JWT_SECRET_KEY` and `OPENAI_SECRET_KEY`
- Don't use a simple string in production! Generate a secure random key:
    ```python
    import secrets
    print(secrets.token_urlsafe(32))
    # something like: xvXM9Z8pQ7YnK4mR2wL5bN6hJ3gF1dS0aT9cE8vP7uO6
    # then in .env file:
    JWT_SECRET_KEY=xvXM9Z8pQ7YnK4mR2wL5bN6hJ3gF1dS0aT9cE8vP7uO6
    ```
- Make sure `.env` is in `.gitignore` 
- In `main.py`, load environment variables:
  ```python
  from dotenv import load_dotenv
  load_dotenv()
  ```

### JWT with Expiration
- Tokens expire after 30 minutes   
- Includes iat (issued at) timestamp

```python
def create_access_token(user_id: int) -> str:
    """Creates a JWT token that expires in 30 minutes"""
    to_encode = {
        "user_id": str(user_id),    # Who this token belongs to
        "exp": datetime.utcnow() + timedelta(minutes=30),# When it expires
        "iat": datetime.utcnow()    # When it was issued
    }
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt

def decode_access_token(token: str) -> Optional[int]:
    """Extracts user_id from token, returns None if invalid/expired"""
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        user_id: str = payload.get("user_id")
        return int(user_id) if user_id else None
    except Exception:
        return None  # Invalid/expired token
```
JWT Token Structure:
```text
eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VyX2lkIjoiMSIsImV4cCI6MTYzOTU...
   ↑ Header                              ↑ Payload                                     
 ↑ Signature
```

### Validation
#### `@field_validator()`
- To validates field (in this case, password) requirements in Pydantic Validation (UserCreate)
- Custom checking logic to a field (password) that is automatically executed when the model is initialized.
  ```python
  from pydantic import field_validator
  class UserCreate(UserBase):
    password: str = Field(..., min_length=6)
    
    @field_validator('password')
    def validate_password(cls, v:str):
        if not any(char.isdigit() for char in v):
            raise ValueError('Password must include one digit')
        return v
  ```
#### `@model_validator()`
For multiple fields validation      
e.g. confirm password == password: 
```python
from pydantic import BaseModel, Field, model_validator
class UserCreate(BaseModel):
    password: str = Field(..., min_length=6)
    confirm_password: str

    @model_validator(mode="after") #Run after all fields have been checked
    def check_passwords_match(self):
        if self.password != self.confirm_password:
            raise ValueError("Password and confirm_password must match")
        return self
```

#### Token Response Schema 
ensures what gets returned when login is successful
```python
class Token(BaseModel):
    access_token: str
```
  
#### Email format validation with `EmailStr`
A specia type provided by Pydantic to verify whether a string is a legitimate email address:   
- must contain @
- a legitimate domain name (e.g., gmail.com)
- standard email format (RFC 5322)
```python
from pydantic import EmailStr
class UserBase(BaseModel):
    email: EmailStr = Field(..., max_length=255)
    username: str = Field(..., min_length=3, max_length=50)
```

### OAuth2 Standard
- Compatible with FastAPI's interactive docs

#### `OAuth2PasswordRequestForm`
FastAPI implements the shape of the OAuth2 Password Grant.  

Not a full OAuth2 provider implementation (no refresh tokens by default, no introspection, etc.), more like “OAuth2-style login form for modern API authentication”: 
- credentials are sent as form-data
- fields are named username, password
- optional scope, client_id, client_secret
- response should return access_token and token_type

```python
# in auth.py:
from fastapi.security import OAuth2PasswordRequestForm
def login(db: DBSession, form_data: OAuth2PasswordRequestForm = Depends()):
  print(form_data.username)
  print(form_data.password)
  ...
```
- `form_data` is an instance of `OAuth2PasswordRequestForm` that FastAPI automatically creates by reading form **fields from** the client request
- The client must send data as **application/x-www-form-urlencoded, not JSON**
- `Depends()` tells **FastAPI**: “Run this dependency before calling my login() function, and give me the result.”
- So **FastAPI**:
  - Looks at the request
  - Extracts form data
  - **Validates it using OAuth2PasswordRequestForm**
  - Creates a form_data object
  - Passes it into your function

#### `OAuth2PasswordBearer`
A security scheme used by FastAPI to tell:
- how clients should send the access token
- where FastAPI should look for the token
- what to display in the automatically generated API docs

It does NOT validate passwords.   
It does NOT check if tokens are valid.   
It only extracts the token from the request.   

```python
# in dependency.py:
from fastapi.security import OAuth2PasswordBearer
password_oauth_scheme = OAuth2PasswordBearer(tokenUrl='/auth/login')
def get_current_user(db: DBSession, token: str = Depends(password_oauth_scheme)):
  ...
```

- Tells FastAPI where to get tokens (the /auth/login endpoint)
- Automatically extracts token from Authorization: Bearer <token> header
- Provides interactive docs with "Authorize" button



### Additional Security Considerations
- Use HTTPS in Production     
  Tokens sent in headers can be intercepted over HTTP

- Refresh Tokens (Future Enhancement)      
  Long-lived refresh tokens for better UX      
  Short-lived access tokens for security       
 
- Token Blacklisting (Future Enhancement)     
  Allow users to revoke tokens (logout)      
  Store invalidated tokens in Redis       

- Rate Limiting (Future Enhancement)       
  Prevent brute force attacks on login

# Async and Await Concepts in FastAPI
Flask is based on **WSGI (Web Server Gateway Interface)**
- Programming Model: **Synchronous** and blocking. 
- Concurrency: Limited, as it relies on threads and doesn't handle many simultaneous connections as efficiently. 
- Protocols: Supports HTTP only.
- Use Case: Best for traditional, I/O-bound or CPU-bound applications that don't require real-time features. 

FastAPI is based on **ASGI (Asynchronous Server Gateway Interface)**
- Programming Model: **Asynchronous** and non-blocking, using the async/await syntax. 
- Concurrency: Excellent for handling thousands of simultaneous connections with a single process. 
- Protocols: Supports HTTP, WebSockets, and can be extended for other protocols. 
- Use Case: Ideal for modern, high-performance web apps needing real-time features, such as chat applications, streaming services, and games.   

So it is called the FAST api.

However, either can handle heavy CPU bound tasks.

## Synchronous vs Asynchronous Route Handlers

**Regular function (synchronous):**
```python
@app.get('/tasks/{task_id}')
def get_task(task_id: int):
    # Blocking operation - server waits here
    return {"task_id": task_id}
```


**Async function:**
```python
@app.get('/tasks/{task_id}')
async def get_task(task_id: int):
    # Non-blocking - server can handle other requests
    result = await some_async_operation()
    return result
```

If you call a blocking (synchronous) function in an async route, it can block the event loop and affect other tasks. The best approach is to use async def with async operations directly. If you must run blocking code, you can wrap it with asyncio.to_thread() that convert synchronous functions to threads.

### When to Use Each?

**Use `def` (synchronous) when:**
- Working with **traditional databases (synchronous SQLAlchemy)**
- Performing CPU-bound operations
- **No I/O** operations or all I/O is synchronous
- Most simple CRUD operations

**Use `async def` (asynchronous) when:**
- Making **external API calls** (like weather forecast APIs)
- Operations that involve waiting for external services
- Using **async database** drivers (**sqlalchemy[asyncio]**)
- Handling file uploads/downloads
- Working with WebSockets

### Example from WeatherForecastTaskMgt
it requires async for the forecast client:
```python
@app.get("/tasks/{task_id}")
async def get_task(
    task_id: int,
    db: Annotated[Session, Depends(get_db)],
    forecast_client: Annotated[ForecastClient, Depends(get_forecast_client)]
):
    # Database query (synchronous)
    task = db.get(Task, task_id)
    
    # External API call (asynchronous)
    weather = await forecast_client.get_weather(task.city)
    
    return {**task.dict(), "weather": weather}
```

- `async def` declares an asynchronous function
- `db.get()` is blocking/synchronous.   
  Since the route is async def, this <u>does block the event loop</u> while querying the database. FastAPI doesn’t automatically run this in a thread. For heavy DB operations, you could wrap it with `await asyncio.to_thread(db.get, Task, task_id)`.
- `forecast_client.get_weather` is async   
- `await` pauses execution until the async operation completes, without blocking the event loop, allowing other requests to be processed
- You can mix sync and async code in async functions
- FastAPI automatically handles the event loop for you

Workflow diagram (conceptually):
1.	HTTP GET /tasks/{task_id} → FastAPI receives request
2.	Resolve dependencies → db session & forecast_client
3.	Execute db.get(Task, task_id) → blocks until DB returns result
4.	Execute await forecast_client.get_weather(task.city) → non-blocking, event loop continues running other tasks
5.	Combine results and return JSON → FastAPI serializes response

## Implementation
> Security:   
0. create `.env` file at root dictory for storing jwt secret key (add file path in .gitignore)
1. in `security_utils.py`, define:
   - password encode and decode methods 
   - JWT access token create and validation methods
2. create new endpoint `auth.py` in ./routers, for user register and login. include the new endpoint in `main.py`
3. `schemas.py`, based on previous, add validation for users and token response
   - for `UserCreate`(used for register), validate field (password) by `@@field_validator()`
   - use `OAuth2PasswordRequestForm` to get the login data, return validated token if successfully login
4. Token required 
   - in flask, a token_required decorator is created and apply in routers
   - in fastapi, use `OAuth2PasswordBearer` to tell it where to get token  
   - declare the token depency in defining function `get_current_user()` in `dependency.py`
   - create a reusable type `CurrentUser = Annotated[User, Depends(get_current_user)]`
   - apply in routers need token_required, like `create_note(current_user:CurrentUser)`

> Async and Await:
0. install async sqlalchemy
1. `database_async.py`: different in URL, create engine, and get session
2. `dependency.py`: change dependency to async to get current user, and database session
3. in endpoints, change to use async dependcies and async def for operation we want it to be async, like list all notes

> Call other service's API (e.g. OpenAI to summarize the note content): 
0. install httpx (support async i/o) and python-dotenv
1. get API secret key and store it in `.env`. To test:
   - put your key in postman's auth bearer token 
   - body: {"model": "gpt-5-nano", "input": "say hello","store": true}
   - POST to https://api.openai.com/v1/responses
2. creat `ai_client.py`
3. apply in endpint, like summarize_note
4. call env variables by `load_dotenv()` in `main.py`

Real-World Migration from old api to new api process:

# FastAPI Unit Testing

## Testing Framework Setup
FastAPI works with standard Python testing tools:
- **pytest**: The main testing framework
- **httpx**: For making test requests (async-capable)

## Basic Test Structure

```python
from fastapi.testclient import TestClient
from main import app

client = TestClient(app)

def test_home():
    response = client.get('/')
    assert response.status_code == 200
    assert response.json() == {'message': 'hello world'}
```



## Testing with Dependencies
When testing endpoints that use dependencies (like database sessions), you can override them:

```python
from fastapi import Depends

# Override database dependency for testing
def override_get_db():
    # Use test database
    db = TestSession()
    try:
        yield db
    finally:
        db.close()

app.dependency_overrides[get_db] = override_get_db
```



## Testing CRUD Operations

```python
def test_create_task():
    response = client.post('/tasks', json={
        "title": "Test task",
        "content": "Test content",
        "city": "Tokyo",
        "user_id": 1
    })
    assert response.status_code == 201
    data = response.json()
    assert data["title"] == "Test task"
    assert "id" in data

def test_get_task_not_found():
    response = client.get('/tasks/9999')
    assert response.status_code == 404
    assert response.json() == {"detail": "Task not found"}
```

## Testing Async Endpoints
For async endpoints, use `pytest-asyncio`:
```python
import pytest
from httpx import AsyncClient

@pytest.mark.asyncio
async def test_async_endpoint():
    async with AsyncClient(app=app, base_url="http://test") as client:
        response = await client.get("/tasks/1")
        assert response.status_code == 200
```

## Custom Error Responses

You can customize validation errors:
```python
from fastapi import HTTPException

@app.get('/tasks/{task_id}')
def get_task(task_id: int, db: Session = Depends(get_db)):
    task = db.get(Task, task_id)
    if not task:
        raise HTTPException(status_code=404, detail="Task not found")
    return task
```

# Key Takeaways

1. **FastAPI is batteries-included**: Comes with validation, documentation, and type checking out of the box
2. **Use async strategically**: Only for I/O-bound operations like external API calls
3. **Pydantic handles validation**: Define schemas once, get validation and documentation automatically
4. **Documentation is free**: Swagger UI and ReDoc are generated automatically from your code
5. **Testing is straightforward**: Use TestClient for sync code, AsyncClient for async code

## Additional Resources

- **FastAPI Official Docs**: https://fastapi.tiangolo.com/
- **Pydantic Documentation**: https://docs.pydantic.dev/
- **SQLModel Documentation**: https://sqlmodel.tiangolo.com/
